In [ ]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib as plt
import gc
import psutil

import Parquet files, convert to Pandas Dataframe, and merge into one file, for ease of exploratory analysis and cleaning

In [ ]:
# y_2023_01 = pq.read_table('./raw_data/yellow_tripdata_2023-01.parquet')
# y_2023_01 = y_2023_01.to_pandas()
# y_2023_02 = pq.read_table('./raw_data/yellow_tripdata_2023-02.parquet')
# y_2023_02 = y_2023_02.to_pandas()
# y_2023_03 = pq.read_table('./raw_data/yellow_tripdata_2023-03.parquet')
# y_2023_03 = y_2023_03.to_pandas()
# y_2023_04 = pq.read_table('./raw_data/yellow_tripdata_2023-04.parquet')
# y_2023_04 = y_2023_04.to_pandas()
# y_2023_05 = pq.read_table('./raw_data/yellow_tripdata_2023-05.parquet')
# y_2023_05 = y_2023_05.to_pandas()
# y_2023_06 = pq.read_table('./raw_data/yellow_tripdata_2023-06.parquet')
# y_2023_06 = y_2023_06.to_pandas()

def get_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    return mem_info.rss / (1024 ** 3)  # Return memory usage in GB

print(f"Initial memory usage: {get_memory_usage():.2f} GB")

y_2023 = pq.read_table('./raw_data/yellow_tripdata_2023-01.parquet')
y_2023 = y_2023.to_pandas()

print(f"Memory usage after 1 file: {get_memory_usage():.2f} GB")

y_2023_02 = pq.read_table('./raw_data/yellow_tripdata_2023-02.parquet')
y_2023_03 = pq.read_table('./raw_data/yellow_tripdata_2023-03.parquet')
y_2023_04 = pq.read_table('./raw_data/yellow_tripdata_2023-04.parquet')
y_2023_05 = pq.read_table('./raw_data/yellow_tripdata_2023-05.parquet')
y_2023_06 = pq.read_table('./raw_data/yellow_tripdata_2023-06.parquet')
y_2023_02 = y_2023_02.to_pandas()
y_2023_03 = y_2023_03.to_pandas()
y_2023_04 = y_2023_04.to_pandas()
y_2023_05 = y_2023_05.to_pandas()
y_2023_06 = y_2023_06.to_pandas()
y_2023 = pd.concat([y_2023, y_2023_02, y_2023_03, y_2023_04, y_2023_05, y_2023_06], axis = 0)

print(f"Memory usage after 6 files and concat: {get_memory_usage():.2f} GB")

del y_2023_02
del y_2023_03
del y_2023_04
del y_2023_05
del y_2023_06

print(f"Memory usage after deleting dataframes: {get_memory_usage():.2f} GB")

gc.collect()

print(f"Memory usage after garbage collection: {get_memory_usage():.2f} GB")

# y_2023_07 = pq.read_table('./raw_data/yellow_tripdata_2023-07.parquet')
# y_2023_07 = y_2023_07.to_pandas()
# y_2023 = pd.concat([y_2023, y_2023_07], axis=0)
# del y_2023_07
# gc.collect()



# y_2023 = pd.concat([y_2023_01, y_2023_02, y_2023_03, y_2023_04, y_2023_05, y_2023_06], axis=0)

In [ ]:
y_2023.info()